In [1]:
import datetime
import sys
import warnings

import jax
import jax.numpy as jnp
from get_model import get_model_and_data  # noqa: F401
from main import run_simple_lmc_numpyro  # noqa: F401
from numpyro.infer import MCMC, NUTS, Predictive  # noqa: F401
from progressive_full_run import run_progressive_logreg


%env JAX_PLATFORM_NAME=cuda
warnings.simplefilter("ignore", FutureWarning)

jnp.set_printoptions(precision=3, suppress=True, threshold=sys.maxsize)
jax.config.update("jax_enable_x64", True)
print(jax.devices("cuda"))

names = [
    "banana",
    "breast_cancer",
    "diabetis",
    "flare_solar",
    "german",
    "heart",
    "image",
    "ringnorm",
    "splice",
    "thyroid",
    "titanic",
    "twonorm",
    "waveform",
]

env: JAX_PLATFORM_NAME=cuda
[CudaDevice(id=0)]


In [2]:
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_filename = f"progressive_results/log_{timestamp}.txt"
with open(log_filename, "w") as f:
    f.write(f"Results for time {timestamp}\n\n")

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
for name in names:
    print(f"==================== {name} ====================")
    run_progressive_logreg(name, log_filename, timestamp)
    print()

==================== image ====================
Data shape: (2086, 18)


ValueError: MultivariateNormal distribution got invalid precision_matrix parameter.